In [1]:
!pip install geemap pycrs earthengine-api pygis cartopy --quiet --no-cache-dir

In [2]:
import geemap
import ee
import matplotlib.pyplot as plt
import numpy as np

from geemap import cartoee
import cartopy.crs as ccrs
import cartopy

import earthaccess

In [3]:
ee.Initialize(project='ee-lc-trends')

General Data Collection

In [4]:
roi = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq('ADM0_NAME', 'Afghanistan'))

L5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
L8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')


In [5]:
def scale_factors (image):
    optical_bands = image.select('SR_B.').multiply(2.75e-05).add(-0.2)
    thermal_bands = image.select('ST_B6').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

def clipping (image):
    return image.clip(roi)

Setting out the Bands

Base Year

In [6]:
# in Landsat 5 (L5)
image_col = L5.filterDate('1990-01-01', '1990-12-31').filterBounds(roi)
image_col = image_col.filter(ee.Filter.lt('CLOUD_COVER', 20))
image_col = image_col.map(scale_factors)
image_col = image_col.map(clipping)

image = image_col.reduce(ee.Reducer.percentile([20]))
image = image.select(['SR_B1_p20', 'SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20', 'SR_B7_p20'])
image = image.rename(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])

ndvi = image.normalizedDifference(['nir', 'red']).rename('ndvi')
image = image.addBands(ndvi)

image = image.select(['green', 'nir', 'swir2', 'ndvi'])
image = image.rename(['green', 'nir', 'swir', 'ndvi'])

image

Buffer Years

In [7]:
def timeseries_L5(startYear, endYear, startMonth, endMonth):
    image_col = L5.filterBounds(roi)
    image_col = image_col.filter(ee.Filter.calendarRange(startYear, endYear, 'year')).filter(ee.Filter.calendarRange(startMonth, endMonth, 'month'))
    image_col = image_col.filter(ee.Filter.lt('CLOUD_COVER', 20))
    image = image_col.reduce(ee.Reducer.percentile([20]))
    image = image.clip(roi)
    image = image.multiply(2.75e-05).add(-0.2)
    image = image.select(['SR_B1_p20', 'SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20'])
    image = image.rename(['blue', 'green', 'red', 'nir', 'swir'])

    kernel = ee.Kernel.square(radius=8)
    kernel_image = image.select('swir').neighborhoodToBands(kernel)
    swir = kernel_image.reduce(ee.Reducer.percentile([20]))
    swir = swir.rename('swir2')
    
    ndvi = image.normalizedDifference(['nir', 'red']).rename('ndvi')
    kernel_image = ndvi.neighborhoodToBands(kernel)
    ndvi = kernel_image.reduce(ee.Reducer.percentile([20]))
    ndvi = ndvi.rename('ndvi')
    
    image = image.addBands(ndvi).addBands(swir)
    image_xx = image.select(['swir2', 'ndvi'])
    return image_xx

def timeseries_L8(startYear, endYear, startMonth, endMonth):
    image_col = L8.filterBounds(roi)
    image_col = image_col.filter(ee.Filter.calendarRange(startYear, endYear, 'year')).filter(ee.Filter.calendarRange(startMonth, endMonth, 'month'))
    image_col = image_col.filter(ee.Filter.lt('CLOUD_COVER', 20))
    image = image_col.reduce(ee.Reducer.percentile([20]))
    image = image.clip(roi)
    image = image.multiply(2.75e-05).add(-0.2)
    image = image.select(['SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20', 'SR_B6_p20'])
    image = image.rename(['blue', 'green', 'red', 'nir', 'swir'])
    ndvi = image.normalizedDifference(['nir', 'red']).rename('ndvi')
    image = image.addBands(ndvi)
    image = image.select(['swir', 'ndvi'])
    return ndvi

Sequence Years Period

In [8]:
image_1 = timeseries_L5(1988, 1992, 1, 2).rename(['swir_1', 'ndvi_1'])
image_2 = timeseries_L5(1988, 1992, 3, 4).rename(['swir_2', 'ndvi_2'])
image_3 = timeseries_L5(1988, 1992, 5, 6).rename(['swir_3', 'ndvi_3'])
image_4 = timeseries_L5(1988, 1992, 7, 8).rename(['swir_4', 'ndvi_4'])
image_5 = timeseries_L5(1988, 1992, 9, 10).rename(['swir_5', 'ndvi_5'])
image_6 = timeseries_L5(1988, 1992, 11, 12).rename(['swir_6', 'ndvi_6'])

image_1

In [9]:
stacked = ee.Image([image, image_1, image_2, image_3, image_4, image_5, image_6])
stacked